In [7]:
import zipfile as zipfile
import pandas as pd
import datetime as dt
import folium

m = folium.Map([-5.1998, -39.2896], zoom_start=7)

print("\nLendo dados das estações pluviometricas...")

for_concat = []
with zipfile.ZipFile('../data/postos.zip', 'r') as t:
    for arq in t.namelist():
        if arq.endswith('.txt'):
            with t.open(arq) as f:
                df = pd.read_csv(f, delimiter=';')
                # Extrair o ID do nome do arquivo e adicionar como uma nova coluna
                df['id'] = arq.split('.')[0]
                for_concat.append(df)

# Exclude empty or all-NA entries before concatenation
for_concat = [
    df for df in for_concat if not df.empty and not df.isna().all().all()
]

registros_chuvas = pd.concat(for_concat)
registros_chuvas.fillna(0, inplace=True)
registros_chuvas.reset_index(drop=True, inplace=True)

# Reordenar as colunas para que 'id' seja a primeira
cols = ['id'] + [col for col in registros_chuvas.columns if col != 'id']
registros_chuvas = registros_chuvas[cols]


Lendo dados das estações pluviometricas...


In [8]:
print("\nFiltrando anos completos...")

#pegar apenas os anos completos
registros_chuvas['Anos'] = registros_chuvas['Anos'].astype(int)
registros_chuvas['Meses'] = registros_chuvas['Meses'].astype(int)
ano = dt.datetime.now().year
Anos_completos = registros_chuvas
Anos_completos = Anos_completos.loc[Anos_completos['Anos'] < ano]

Anos_completos = Anos_completos = Anos_completos.groupby('id').first().reset_index()
Anos_completos


Filtrando anos completos...


In [12]:
for idx, row in Anos_completos.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Postos'],
        icon=folium.Icon(icon="cloud"),
    ).add_to(m)

In [ ]:
m.save('../data/mapa.html')

FileNotFoundError: [Errno 2] No such file or directory: './data/mapa.html'